Here I combine two csv files and perform some exploratory data analyses.

In [1]:
import pandas as pd

Next, the two csv files are read into memory.

In [2]:
behav_data = pd.read_csv("behav_combined_out.csv", encoding = 'latin-1')
videogr_data = pd.read_csv("cops_measured_video_comments.csv", encoding = 'latin-1')

How many copepods have a behavior measurement?

In [3]:
behav_data['cop_name'].nunique()

689

In [4]:
videogr_data = videogr_data.rename(columns={'cop':'cop_name'})
videogr_data['cop_name'].nunique()

686

Pretty similar, but a few are missing from the videography data. Just three. Keep in mind.

In [5]:
for x in range(0, behav_data['cop_name'].nunique() ):
    cop = behav_data['cop_name'].unique()[x]
    if cop not in videogr_data['cop_name'].unique():
        print(cop)

39_1B
50_6D
66_1B


Merge the two data frames

In [6]:
combined_data = pd.merge(behav_data, 
                         videogr_data[['fname', 'cop_name', 'day', 'by', 'Ok?', 'remarks']],
                         how = 'left', on = 'fname')

Count problematic cases.

In [7]:
(combined_data.groupby(by = 'fname')
 .size())

fname
01_1D_11    62
01_1D_13    62
01_1D_15    62
01_1D_17    62
01_1D_5     62
01_1D_7     62
01_1D_9     62
01_2B_11    62
01_2B_5     62
01_2B_7     62
01_2B_9     62
01_2C_11    62
01_2C_13    62
01_2C_15    62
01_2C_17    62
01_2C_5     62
01_2C_7     62
01_2C_9     62
01_3D_11    62
01_3D_13    62
01_3D_15    62
01_3D_17    62
01_3D_5     62
01_3D_7     62
01_3D_9     62
01_4A_11    62
01_4A_13    62
01_4A_15    62
01_4A_17    62
01_4A_5     62
            ..
74_2B_5     62
74_2B_7     62
74_2B_9     62
74_3A_11    62
74_3A_13    63
74_3A_15    62
74_3A_17    62
74_3A_5     62
74_3A_7     62
74_3A_9     62
74_3D_11    62
74_3D_13    63
74_3D_15    62
74_3D_17    62
74_3D_5     62
74_3D_7     62
74_3D_9     62
74_4A_11    62
74_4A_13    63
74_4A_15    62
74_4A_5     62
74_4A_7     62
74_4A_9     62
74_5D_11    62
74_5D_13    63
74_5D_15    62
74_5D_17    62
74_5D_5     62
74_5D_7     62
74_5D_9     62
Length: 4566, dtype: int64

In [8]:
(combined_data.groupby(by = 'fname')
 .mean())

,Unnamed: 0,Slice n°,X,Y,Distance,Pixel Value,ok_col_names,ok_col_num,ok_row_num,day_x,day_y,Ok?
fname,,,,,,,,,,,,
01_1D_11,30.5,31.5,52.564516,50.290323,2.738532,149.064516,1.0,0.0,0.0,11.0,11.0,0.0
01_1D_13,30.5,31.5,49.370968,38.806452,1.102452,144.548387,0.0,0.0,0.0,13.0,13.0,0.0
01_1D_15,30.5,31.5,45.048387,45.258065,1.628435,130.806452,0.0,0.0,0.0,15.0,15.0,0.0
01_1D_17,30.5,31.5,48.516129,45.967742,1.478387,128.370968,0.0,0.0,0.0,17.0,17.0,0.0
01_1D_5,30.5,31.5,59.387097,62.741935,1.125161,132.967742,0.0,0.0,0.0,5.0,5.0,0.0
01_1D_7,30.5,31.5,64.983871,58.451613,2.696403,148.322581,0.0,0.0,0.0,7.0,7.0,0.0
01_1D_9,30.5,31.5,41.564516,51.387097,1.934790,155.000000,0.0,0.0,0.0,9.0,9.0,0.0
01_2B_11,30.5,31.5,56.000000,57.274194,1.409532,178.145161,1.0,0.0,0.0,11.0,11.0,0.0
01_2B_5,30.5,31.5,68.435484,67.887097,1.852016,136.870968,0.0,0.0,0.0,5.0,5.0,0.0
